In [ ]:
import tensorflow_datasets as tfds

In [7]:
train, val, test=tfds.load('celeb_a', as_supervised=True, shuffle_files=True, split=['train', 'validation', 'test'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

NonMatchingChecksumError: Artifact https://drive.google.com/uc?export=download&id=0B7EVK8r0v71pZjFTYXZWM3FlRnM, downloaded to /home/phucphuong2112/tensorflow_datasets/downloads/celeb_a/ucexport_download_id_0B7EVK8r0v71pZjFTYXZWM3FlDDaXUAQO8EGH_a7VqGNLRtW52mva1LzDrb-V723OQN8.tmp.10e5f8e45f734a14b414edfa301fda67/download, has wrong checksum:
* Expected: UrlInfo(size=1.34 GiB, checksum='46fb89443c578308acf364d7d379fe1b9efb793042c0af734b6112e4fd3a8c74', filename='img_align_celeba.zip')
* Got: UrlInfo(size=1.96 KiB, checksum='e82b2fd3562c5a3728994db8b76ed12d1180ace22b3bd35fa0e90cb37712c918', filename='download')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror